# Topic Canonicalization Workflow
Consolidate NOVEL topics into the registry using embeddings + LLM review.

In [7]:
import polars as pl
from pathlib import Path
import json
from datetime import datetime
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from utils.llm_client import LLMClient

DATA_DIR = Path('../data')
REGISTRY_PATH = Path('../registry/topic_registry.json')
MODEL_NAME = 'all-MiniLM-L6-v2'
SIM_THRESHOLD = 0.82
LLM_PROVIDER = 'megallm'
LLM_MODEL = 'gpt-4o-mini'

print('✓ Environment ready')


✓ Environment ready


## Load Registry and Novel Candidates

In [2]:

with open(REGISTRY_PATH) as f:
    registry = json.load(f)

registry_version = registry.get('version', 'unknown')
registry_created = registry.get('created_at')
existing_topics = registry['topics']
print(f"Loaded {len(existing_topics)} registry topics (version {registry_version}, created {registry_created})")

novel_frames = []
for path in sorted((DATA_DIR / 'daily_labels').glob('labels_*.parquet')):
    df = pl.read_parquet(path)
    novel_subset = df.filter((pl.col('topic_id') == 'NOVEL') & pl.col('novel_label').is_not_null())
    if len(novel_subset) > 0:
        novel_frames.append(novel_subset)

if not novel_frames:
    raise ValueError('No NOVEL topics found to canonicalize.')

novel_df = pl.concat(novel_frames).with_columns([
    pl.col('created_at').dt.convert_time_zone('Asia/Kolkata'),
]).unique(subset=['novel_label'])
print(f'Found {len(novel_df)} unique novel candidates')


Loaded 32 registry topics (version 001, created 2025-10-27T00:00:00+05:30)
Found 1028 unique novel candidates


## Summarize Novel Candidates

In [3]:

# Summarize novel topics by frequency and recency
novel_summary = novel_df.group_by('novel_label').agg([
    pl.len().alias('review_count'),
    pl.first('novel_rationale').alias('sample_rationale'),
    pl.min('created_at').alias('first_seen'),
    pl.max('created_at').alias('last_seen'),
]).sort('review_count', descending=True)

print(f"✓ Aggregated {len(novel_summary)} novel topic candidates")
summary_path = DATA_DIR / 'novel_topic_summary.parquet'
novel_summary.write_parquet(summary_path)
print(f"  Saved summary to {summary_path}")

try:
    display(novel_summary.head(10))
except NameError:
    print(novel_summary.head(10))


✓ Aggregated 1028 novel topic candidates
  Saved summary to ../data/novel_topic_summary.parquet


novel_label,review_count,sample_rationale,first_seen,last_seen
str,u32,str,"datetime[μs, Asia/Kolkata]","datetime[μs, Asia/Kolkata]"
"""Request for delivery""",1,"""The review is a simple request…",2025-10-25 16:29:17 IST,2025-10-25 16:29:17 IST
"""Unrelated Review Content""",1,"""The review does not relate to …",2025-10-11 17:24:26 IST,2025-10-11 17:24:26 IST
"""Disappointment with medicine o…",1,"""The review expresses disappoin…",2025-10-18 14:24:24 IST,2025-10-18 14:24:24 IST
"""Request for better offers""",1,"""The review suggests improving …",2025-09-29 17:57:36 IST,2025-09-29 17:57:36 IST
"""Awesome Offers""",1,"""The review mentions awesome of…",2025-10-05 09:18:24 IST,2025-10-05 09:18:24 IST
"""Poor Delivery Experience""",1,"""The term 'bakwaas delivery' in…",2025-10-06 10:11:32 IST,2025-10-06 10:11:32 IST
"""Positive feedback on deals""",1,"""The review mentions 'great dea…",2025-10-20 16:39:06 IST,2025-10-20 16:39:06 IST
"""App Deletion Experience""",1,"""User deleted the app due to di…",2025-10-25 14:27:35 IST,2025-10-25 14:27:35 IST
"""Lack of offers""",1,"""The review mentions a lack of …",2025-10-08 05:33:39 IST,2025-10-08 05:33:39 IST


## Compute Embeddings and Similarities

In [5]:
import time

model = SentenceTransformer(MODEL_NAME)
print(f"✓ Loaded embedding model {MODEL_NAME}")

# Build the registry corpus
registry_corpus = [topic['name'] + ' ' + topic['definition'] for topic in existing_topics]
print(f"[LOG] Constructed registry_corpus: {len(registry_corpus)} entries")

# Encode registry topics
start_time = time.time()
registry_matrix = model.encode(registry_corpus, show_progress_bar=True)
elapsed = time.time() - start_time
print(f"[LOG] Encoded registry_corpus into registry_matrix with shape {registry_matrix.shape} in {elapsed:.2f}s")

# Prepare novel texts
novel_texts = [
    f"{row['novel_label']} :: {row['novel_rationale']}" if row['novel_rationale'] else row['novel_label']
    for row in novel_df.iter_rows(named=True)
]
print(f"[LOG] Constructed novel_texts: {len(novel_texts)} entries")

# Encode novel texts
start_time = time.time()
novel_matrix = model.encode(novel_texts, show_progress_bar=True)
elapsed = time.time() - start_time
print(f"[LOG] Encoded novel_texts into novel_matrix with shape {novel_matrix.shape} in {elapsed:.2f}s")

# Compute cosine similarities
start_time = time.time()
similarities = cosine_similarity(novel_matrix, registry_matrix)
elapsed = time.time() - start_time
print(f"[LOG] Computed cosine_similarity matrix with shape {similarities.shape} in {elapsed:.2f}s")

# Build candidate matches
candidate_matches = []
print("[LOG] Beginning to iterate through novel rows for matching…")
for idx, novel_row in enumerate(novel_df.iter_rows(named=True)):
    sim_row = similarities[idx]
    best_idx = int(sim_row.argmax())
    best_score = float(sim_row[best_idx])
    registry_topic = existing_topics[best_idx]
    candidate = {
        'novel_label': novel_row['novel_label'],
        'novel_rationale': novel_row['novel_rationale'],
        'first_seen': novel_row['created_at'],
        'best_topic_id': registry_topic['id'],
        'best_topic_name': registry_topic['name'],
        'similarity': best_score
    }
    candidate_matches.append(candidate)

    # Detailed log per row (optional: comment out for huge datasets)
    print(f"[LOG][row {idx}] novel_label='{candidate['novel_label']}' → "
          f"best_topic_id='{candidate['best_topic_id']}', "
          f"best_topic_name='{candidate['best_topic_name']}', "
          f"similarity={candidate['similarity']:.4f}")

print(f"[LOG] Built {len(candidate_matches)} candidate matches")

candidate_df = pl.DataFrame(candidate_matches).sort('similarity', descending=True)
print(f"✓ Candidate matches computed ({len(candidate_df)} rows)")

try:
    display(candidate_df.head(10))
except NameError:
    print(candidate_df.head(10))


✓ Loaded embedding model all-MiniLM-L6-v2
[LOG] Constructed registry_corpus: 32 entries


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[LOG] Encoded registry_corpus into registry_matrix with shape (32, 384) in 0.20s
[LOG] Constructed novel_texts: 1028 entries


Batches:   0%|          | 0/33 [00:00<?, ?it/s]

[LOG] Encoded novel_texts into novel_matrix with shape (1028, 384) in 0.35s
[LOG] Computed cosine_similarity matrix with shape (1028, 32) in 0.00s
[LOG] Beginning to iterate through novel rows for matching…
[LOG][row 0] novel_label='Restaurant policy on offers' → best_topic_id='LIMITED_OPTIONS', best_topic_name='Limited Options', similarity=0.4673
[LOG][row 1] novel_label='Weird payment option' → best_topic_id='MONEY_ALREADY_EATEN', best_topic_name='Money Already Eaten', similarity=0.4296
[LOG][row 2] novel_label='Unclear review context' → best_topic_id='NO_RESPONSE_COMPLAINT', best_topic_name='No Response to Complaint', similarity=0.2047
[LOG][row 3] novel_label='Promotional Offer Mentioned' → best_topic_id='OVERPRICED_ITEMS', best_topic_name='Overpriced Items', similarity=0.2944
[LOG][row 4] novel_label='Service Issue' → best_topic_id='POSITIVE_EXPERIENCE', best_topic_name='Positive Experience', similarity=0.4307
[LOG][row 5] novel_label='Hunger Issue' → best_topic_id='POOR_QUALITY',

novel_label,novel_rationale,first_seen,best_topic_id,best_topic_name,similarity
str,str,"datetime[μs, Asia/Kolkata]",str,str,f64
"""Positive service feedback""","""The review expresses satisfact…",2025-10-04 16:13:01 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.840523
"""Generic positive feedback""","""The review expresses a positiv…",2025-09-28 05:16:42 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.827888
"""Best service feedback""","""The review expresses a positiv…",2025-10-20 04:11:57 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.753112
"""Delivery Speed Concern""","""The review expresses dissatisf…",2025-10-15 09:30:44 IST,"""FAST_DELIVERY""","""Fast Delivery""",0.730695
"""Positive feedback about servic…","""The review expresses a positiv…",2025-10-04 16:00:59 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.724916
"""Order not received""","""The review indicates waiting f…",2025-10-18 16:13:18 IST,"""NO_DELIVERY_YET""","""No Delivery Yet""",0.712767
"""Low Price Concern""","""The review mentions low price …",2025-09-30 16:34:57 IST,"""POOR_QUALITY""","""Poor Quality""",0.708216
"""Generic Positive Feedback""","""The review expresses a positiv…",2025-09-28 07:02:46 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.69851
"""Slow Delivery""","""The review indicates a delay w…",2025-10-22 14:19:24 IST,"""FAST_DELIVERY""","""Fast Delivery""",0.6936


## Resolve Ambiguous Candidates with LLM

In [10]:
import time

for size in [5, 10, 20, 50, 100]:
    start = time.time()
    llm.batch_complete(prompts[:size], batch_size=size)
    print(f"Batch {size}: {time.time()-start:.2f}s")


Processing: 100%|█████████████████████████████████| 1/1 [00:00<00:00,  3.97it/s]


Batch 5: 0.25s


Processing: 100%|█████████████████████████████████| 1/1 [00:00<00:00,  1.99it/s]


Batch 10: 0.50s


Processing: 100%|█████████████████████████████████| 1/1 [00:01<00:00,  1.01s/it]


Batch 20: 1.01s


Processing: 100%|█████████████████████████████████| 1/1 [00:02<00:00,  2.51s/it]


Batch 50: 2.51s


Processing: 100%|█████████████████████████████████| 1/1 [00:41<00:00, 41.02s/it]

Batch 100: 41.02s


In [12]:
import concurrent.futures
import polars as pl
import time

llm = LLMClient(provider=LLM_PROVIDER, model=LLM_MODEL)
print(f"✓ Ready LLM adjudicator ({LLM_PROVIDER}/{LLM_MODEL})")

actions, prompts = [], []

# Build prompt list
for row in candidate_df.iter_rows(named=True):
    if row['similarity'] >= SIM_THRESHOLD:
        actions.append({**row, 'action': 'merge', 'notes': 'auto-merge via embedding score'})
    else:
        prompts.append({
            'system_prompt': 'You are a taxonomy expert consolidating duplicate topics. Only reply with strict JSON.',
            'user_prompt': f"""Topic A: {row['novel_label']} ({row['novel_rationale']})
Topic B: {row['best_topic_name']}
Return JSON: {{"action":"merge|new","label":"<final>","notes":"<reason>"}}""",
            'temperature': 0.0,
            'response_format': 'json'
        })

# Function to handle one batch safely
def process_batch(batch, batch_id):
    try:
        start = time.time()
        result = llm.batch_complete(batch, batch_size=10)
        print(f"✓ Batch {batch_id} done in {time.time() - start:.2f}s ({len(batch)} prompts)")
        return result
    except Exception as e:
        print(f"⚠️ Batch {batch_id} failed: {e}")
        return [None] * len(batch)

# Run batches in parallel
responses = []
if prompts:
    batches = [prompts[i:i+10] for i in range(0, len(prompts), 10)]
    print(f"🚀 Total batches: {len(batches)} (running with max_workers=20)")

    start = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        future_to_id = {executor.submit(process_batch, b, i+1): i+1 for i, b in enumerate(batches)}
        for future in concurrent.futures.as_completed(future_to_id):
            batch_id = future_to_id[future]
            try:
                res = future.result()
                if res:
                    responses.extend(res)
            except Exception as e:
                print(f"⚠️ Batch {batch_id} error: {e}")
    print(f"✅ All batches completed in {time.time() - start:.2f}s")
else:
    responses = []

# Merge responses back to dataframe
for resp, row in zip(
    responses,
    (r for r in candidate_df.iter_rows(named=True) if r['similarity'] < SIM_THRESHOLD)
):
    if resp is None:
        actions.append({**row, 'action': 'review', 'notes': 'LLM response missing'})
        continue
    actions.append({
        **row,
        'action': resp.get('action', 'review'),
        'notes': resp.get('notes'),
        'llm_label': resp.get('label')
    })

actions_df = pl.DataFrame(actions)
print(f"✓ Proposed actions: {actions_df.height} rows")
try:
    display(actions_df.head(10))
except NameError:
    print(actions_df.head(10))


✓ Initialized megallm client with model gpt-4o-mini
✓ Ready LLM adjudicator (megallm/gpt-4o-mini)
🚀 Total batches: 103 (running with max_workers=20)



Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]

Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]


Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]



Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]





Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]




Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]






Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]









Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]








Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]











Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]










Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]







Processing:   0%|                         

✓ Batch 2 done in 27.25s (10 prompts)


Processing:   0%|                                         | 0/1 [00:00<?, ?it/s]

















Processing: 100%|█████████████████████████████████| 1/1 [00:32<00:00, 32.82s/it]


✓ Batch 19 done in 32.87s (10 prompts)


...














Processing: 100%|█████████████████████████████████| 1/1 [00:30<00:00, 30.14s/it]


✓ Batch 99 done in 30.14s (10 prompts)




















Processing: 100%|█████████████████████████████████| 1/1 [00:33<00:00, 33.44s/it]

✓ Batch 101 done in 33.44s (10 prompts)
✅ All batches completed in 178.55s
✓ Proposed actions: 1028 rows


novel_label,novel_rationale,first_seen,best_topic_id,best_topic_name,similarity,action,notes,llm_label
str,str,"datetime[μs, Asia/Kolkata]",str,str,f64,str,str,str
"""Positive service feedback""","""The review expresses satisfact…",2025-10-04 16:13:01 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.840523,"""merge""","""auto-merge via embedding score""",null
"""Generic positive feedback""","""The review expresses a positiv…",2025-09-28 05:16:42 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.827888,"""merge""","""auto-merge via embedding score""",null
"""Best service feedback""","""The review expresses a positiv…",2025-10-20 04:11:57 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.753112,"""merge""","""Both topics express positive s…","""General Positive Feedback"""
"""Delivery Speed Concern""","""The review expresses dissatisf…",2025-10-15 09:30:44 IST,"""FAST_DELIVERY""","""Fast Delivery""",0.730695,"""merge""","""Consolidated feedback related …","""Service Feedback"""
"""Positive feedback about servic…","""The review expresses a positiv…",2025-10-04 16:00:59 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.724916,"""merge""","""Both topics address aspects of…","""Food Quality"""
"""Order not received""","""The review indicates waiting f…",2025-10-18 16:13:18 IST,"""NO_DELIVERY_YET""","""No Delivery Yet""",0.712767,"""merge""","""Both topics express a positive…","""Positive Feedback on Food Qual…"
"""Low Price Concern""","""The review mentions low price …",2025-09-30 16:34:57 IST,"""POOR_QUALITY""","""Poor Quality""",0.708216,"""new""","""This topic highlights excellen…","""Positive Offers Feedback"""
"""Generic Positive Feedback""","""The review expresses a positiv…",2025-09-28 07:02:46 IST,"""POSITIVE_EXPERIENCE""","""Positive Experience""",0.69851,"""merge""","""Both topics express a general …","""App inquiry"""
"""Slow Delivery""","""The review indicates a delay w…",2025-10-22 14:19:24 IST,"""FAST_DELIVERY""","""Fast Delivery""",0.6936,"""merge""","""Both topics express positive s…","""Positive Feedback and Experien…"


## Write Registry Update Draft

In [13]:

updates = []
timestamp = datetime.utcnow().isoformat() + 'Z'
for row in actions_df.iter_rows(named=True):
    updates.append({
        'timestamp': timestamp,
        'novel_label': row['novel_label'],
        'target_topic_id': row['best_topic_id'],
        'target_topic_name': row['best_topic_name'],
        'similarity': row['similarity'],
        'action': row['action'],
        'notes': row.get('notes'),
        'llm_label': row.get('llm_label'),
        'first_seen': row.get('first_seen'),
    })

updates_path = DATA_DIR / 'registry_updates.json'
with updates_path.open('w') as f:
    json.dump(updates, f, indent=2, default=str)
print(f'✓ Proposed updates written to {updates_path}')

review_path = DATA_DIR / 'registry_updates_preview.csv'
actions_df.write_csv(review_path)
print(f'  Preview CSV written to {review_path}')


✓ Proposed updates written to ../data/registry_updates.json
  Preview CSV written to ../data/registry_updates_preview.csv


## Manual Follow-up
- Review `data/registry_updates.json`
- Apply merges/new topics in `registry/topic_registry.json`
- Re-run topic routing if registry changes materially
